In [31]:
import sys, os
import serial
import time
import numpy as np
import matplotlib.pyplot as plt
# import nbimporter
from picoammeter_control import (
    send_cmd,
    query,
    init_cmds
)
sys.path.insert(0, os.path.abspath('.'))
sys.path.insert(0, os.path.abspath('filter_wheel_control'))
from filter_wheel_control.wheel_controller import (
    list_devices,
    open_device,
    get_position,
    get_position_count,
    set_position,
    close_device
)

In [21]:
print("Finding filter wheel devices...")
devices = list_devices()
if not devices:
    print("No filter wheel devices found.")
    sys.exit(1)

print("Found devices:")
for idx, dev in enumerate(devices):
    print(f" [{idx}] {dev}")

# init filter wheels to empty pos
for i in range(len(devices)):
    handle = open_device(i)
    max_pos = get_position_count(handle)
    if max_pos != 0:
        get_position(handle)
        set_position(handle, 1)
        get_position(handle)
    close_device(handle)

Finding filter wheel devices...
No filter wheel devices found.


SystemExit: 1

In [27]:
# Open serial connection
ser = serial.Serial(
    port='COM3',         # Replace with your actual port
    baudrate=9600,
    bytesize=serial.EIGHTBITS,
    parity=serial.PARITY_NONE,
    stopbits=serial.STOPBITS_ONE,
    timeout=2,
    xonxoff=False,
    rtscts=False,
    dsrdtr=False
)

init_cmds()

read_time = .5 # time in minutes to read for
num_of_reads = round(417*read_time)
readings = np.zeros(num_of_reads)

# Trigger a read and fetch result
for i in range(num_of_reads):
    time.sleep(0.001)
    current = query('READ?')
    readings[i] = current

    if i == num_of_reads//2:
        print("switching filters")
        for i in range(len(devices)):
            handle = open_device(i)
            max_pos = get_position_count(handle)
            if max_pos == 12:
                set_position(handle, 4)
            close_device(handle)
        print("finished switching")

ser.close()

AttributeError: module 'serial' has no attribute 'Serial'

In [24]:
plt.scatter(range(num_of_reads), readings)
plt.show()
# print(f"mean: {np.mean(readings)}")

NameError: name 'num_of_reads' is not defined